In [13]:
import sys
sys.path.append('/Users/rkam/projs/algotrade')
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from src.dbmanager import DBManager
from src.trend_following_rules.crossover_rule import CrossoverRule

tqdm.pandas()

In [14]:
db = DBManager(path='/Users/rkam/projs/algotrade/data/db/tr.db')

In [15]:
prices = db.read(
    """
    select
        *
    from
        tr
    """
)
prices.columns = ['t', 'd', 'p', 'v']
prices['t'] = pd.to_datetime(prices['t'], format='mixed')
prices['t'] = prices['t'].dt.tz_localize(None)
prices = prices.iloc[:40000]

## Crossover Rule

In [10]:
trials = pd.read_csv('/Users/rkam/projs/algotrade/data/interim/trials_crossover.csv', index_col=0)
trials = trials.sort_values('value', ascending=False)
trials = trials.drop(['number', 'datetime_start', 'datetime_complete', 'duration', 'state'], axis=1)
trials = trials.drop_duplicates().reset_index(drop=True)
trials

,value,params_eps,params_long_window_diff,params_ma_long_type,params_ma_short_type,params_sl,params_tp,params_window_short
0,-0.767056,0.37,800,ema,ema,-0.70,0.47,600
1,-1.091136,0.40,800,sma,ema,-0.45,0.52,700
2,-1.118600,0.45,400,lma,ema,-0.52,0.23,1000
3,-1.444400,0.30,400,ema,lma,-0.40,0.47,700
4,-2.388024,0.08,400,ema,lma,-0.83,0.49,900
5,-3.028120,0.49,300,ema,lma,-0.18,0.35,400
6,-5.094640,0.19,900,sma,lma,-0.74,0.42,200
7,-10.000000,0.47,700,lma,sma,-0.55,0.23,700
8,-10.000000,0.35,800,sma,lma,-0.31,0.74,800
9,-12.712664,0.03,300,ema,sma,-0.59,0.86,400


In [ ]:
total_profit = 0
for day in prices['t'].dt.date.unique():

    pr = prices[prices['t'].dt.date == day]
    start = datetime(year=day.year, month=day.month, day=day.day, hour=10)
    end = datetime(year=day.year, month=day.month, day=day.day, hour=18)
    pr = pr[(pr['t'] >= start) & (pr['t'] <= end)].reset_index(drop=True)

    cr = CrossoverRule(
        short_long_ma=['sma', 'ema'],
        window_short=400, window_long=400+300,
        eps=0.03, sl=-0.59, tp=0.86
    )
    for i in tqdm(range(0, pr.shape[0])):
        p = pr['p'].iloc[i]
        signal = cr.get_signal(p)
        if signal is not None and signal['action'] == 'close':
            total_profit += signal['profit_current']

